# Project 1 — Mini NGS Pipeline (Jupyter Edition)

**Goal:** Minimal end‑to‑end NGS workflow: **QC → Align → Variant Call → Load VCF into SQLite** and query.

**Tools:** FastQC, BWA, SAMtools, FreeBayes, bgzip/tabix (htslib), Python (sqlite3, pandas).

**Files:** lambda_virus.fa (https://github.com/BenLangmead/bowtie2/blob/master/example/reference/lambda_virus.fa)

reads_1.fq
(https://github.com/BenLangmead/bowtie2/blob/master/example/reads/reads_1.fq)

reads_2.fq
(https://github.com/BenLangmead/bowtie2/blob/master/example/reads/reads_2.fq)


## Create project structure


In [6]:
from pathlib import Path

ROOT = Path.cwd() / "project1"
DATA = ROOT / "data"
REF  = ROOT / "ref"
OUT  = ROOT / "results"
QC   = ROOT / "qc"
SQL  = ROOT / "sql"

for p in [DATA, REF, OUT, QC, SQL]:
    p.mkdir(parents=True, exist_ok=True)

print("Project root:", ROOT)
for p in [DATA, REF, OUT, QC, SQL]:
    print(" -", p)


Project root: C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1
 - C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\data
 - C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\ref
 - C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results
 - C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\qc
 - C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\sql



## 1) Quality Control — FastQC
Outputs reports into `qc/`.


In [7]:

import subprocess
from pathlib import Path

def run(cmd: str):
    print("$", cmd)
    return subprocess.call(cmd, shell=True)

reads_1 = DATA / "reads_1.fq"
assert reads_1.exists(), "reads_1.fq not found. Run the data-prep cells first."

rc = run(f'conda run -n caris-mini fastqc -o "{QC}" "{reads_1}"')
print("FastQC exit code:", rc)
print("QC dir:", QC)


$ conda run -n caris-mini fastqc -o "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\qc" "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\data\reads_1.fq"
FastQC exit code: 1
QC dir: C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\qc



## 2) Alignment — BWA (SAM → BAM → sorted BAM → index)


In [8]:

ref_fa = REF / "lambda_virus.fa"
aln_sam = OUT / "aln.sam"
aln_bam = OUT / "aln.bam"
aln_sorted = OUT / "aln.sorted.bam"

assert ref_fa.exists(), "Reference FASTA missing."

# Index reference
run(f'conda run -n caris-mini bwa index "{ref_fa}"')

# Align to SAM
run(f'conda run -n caris-mini bwa mem -t 2 "{ref_fa}" "{reads_1}" > "{aln_sam}"')

# SAM -> BAM
run(f'conda run -n caris-mini samtools view -bS "{aln_sam}" > "{aln_bam}"')

# Sort BAM and replace
run(f'conda run -n caris-mini samtools sort -o "{aln_sorted}" "{aln_bam}"')
run(f'mv "{aln_sorted}" "{aln_bam}"')

# Index BAM
run(f'conda run -n caris-mini samtools index "{aln_bam}"')

print("BAM exists:", aln_bam.exists(), aln_bam)
print("BAI exists:", Path(str(aln_bam)+".bai").exists())


$ conda run -n caris-mini bwa index "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\ref\lambda_virus.fa"
$ conda run -n caris-mini bwa mem -t 2 "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\ref\lambda_virus.fa" "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\data\reads_1.fq" > "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\aln.sam"
$ conda run -n caris-mini samtools view -bS "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\aln.sam" > "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\aln.bam"
$ conda run -n caris-mini samtools sort -o "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\aln.sorted.bam" "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\aln.bam"
$ mv "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\aln.sorted.bam" "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\aln.bam"
$ conda


## 3) Variant Calling — FreeBayes (+ bgzip & tabix)


In [9]:
vcf = OUT / "variants.vcf"
vcf_gz = OUT / "variants.vcf.gz"

# Call variants
run(f'conda run -n caris-mini freebayes -f "{ref_fa}" "{aln_bam}" > "{vcf}"')

# Compress + index
run(f'conda run -n caris-mini bgzip -f "{vcf}"')
run(f'conda run -n caris-mini tabix -f "{vcf_gz}"')

print("VCF GZ:", vcf_gz.exists(), vcf_gz)
print("TBI:", Path(str(vcf_gz)+".tbi").exists())


$ conda run -n caris-mini freebayes -f "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\ref\lambda_virus.fa" "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\aln.bam" > "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\variants.vcf"
$ conda run -n caris-mini bgzip -f "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\variants.vcf"
$ conda run -n caris-mini tabix -f "C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\variants.vcf.gz"
VCF GZ: False C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\project1\results\variants.vcf.gz
TBI: False



## 4) export tidy CSV / Excel

Parse VCF → DataFrame → save CSV & Excel

In [10]:
from pathlib import Path
import pandas as pd, gzip

ROOT = Path.cwd() / "caris_project1"
OUT  = ROOT / "results"

vcf_path = OUT / "variants.vcf"       # created earlier
assert vcf_path.exists(), "variants.vcf not found. Run the calling step first."

records = []
with open(vcf_path, "rt", encoding="utf-8") as fh:
    for line in fh:
        if line.startswith("#"):
            continue
        parts = line.rstrip("\n").split("\t")
        if len(parts) < 8:
            continue  # skip malformed/blank rows
        chrom, pos, vid, ref, alt, qual, flt, info = line.rstrip("\n").split("\t")[:8]
        first_alt = alt.split(",")[0]
        records.append({
            "chrom": chrom,
            "pos": int(pos),
            "id": None if vid == "." else vid,
            "ref": ref,
            "alt": alt,
            "qual": None if qual == "." else float(qual),
            "filter": flt,
            "info": info,
            "type": "SNP" if len(ref) == 1 and len(first_alt) == 1 else "INDEL",
        })

# Ensure columns exist even if empty
cols = ["chrom","pos","id","ref","alt","qual","filter","info","type"]
df = pd.DataFrame.from_records(records, columns=cols)
print("Parsed variants:", len(df))
display(df.head(10))


##SAVE FILES

csv_path  = OUT / "variants.csv"
xlsx_path = OUT / "variants.xlsx"

df.to_csv(csv_path, index=False)

# If Excel export errors, install openpyxl in your env: 
#   conda run -n caris-mini conda install -y -c conda-forge openpyxl
with pd.ExcelWriter(xlsx_path, engine="openpyxl") as xw:
    df.to_excel(xw, sheet_name="variants", index=False)

    # Only build extra sheets if we have rows
    if not df.empty:
        top = df.sort_values("qual", ascending=False, na_position="last").head(50)
        top.to_excel(xw, sheet_name="top_variants", index=False)
        chrom_counts = df["chrom"].value_counts(dropna=False).rename_axis("chrom").reset_index(name="n_variants")
        chrom_counts.to_excel(xw, sheet_name="chrom_counts", index=False)
        summary = pd.DataFrame({
            "metric": ["n_variants", "n_snp", "n_indel", "qual_mean", "qual_median"],
            "value": [len(df),
                      int((df["type"] == "SNP").sum()),
                      int((df["type"] == "INDEL").sum()),
                      float(df["qual"].mean(skipna=True)),
                      float(df["qual"].median(skipna=True))]
        })
    else:
        # Empty summary if no variants
        summary = pd.DataFrame({
            "metric": ["n_variants", "n_snp", "n_indel", "qual_mean", "qual_median"],
            "value": [0, 0, 0, None, None]
        })
    summary.to_excel(xw, sheet_name="summary", index=False)

print("Wrote:", csv_path)
print("Wrote:", xlsx_path)

Parsed variants: 0


,chrom,pos,id,ref,alt,qual,filter,info,type


Wrote: C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\caris_project1\results\variants.csv
Wrote: C:\Users\Jisoo Chae\Self project\2 Mini NGS pipeline\caris_project1\results\variants.xlsx
